# Extração das Features para Tipificação do Fogo

Código referente à extração de features necessárias para a tipificação do fogo. Assim, começamos importando as seguintes bibliotecas que serão utilizadas durante todo o processo:

<dd>• OS (Operating System)</dd>
<dd>• GeoPandas</dd>
<dd>• Datetime</dd>
<dd>• Rasterio</dd>
<dd>• PyGeos</dd>
<dd>• Numpy</dd>

É recomendado, também, que seja utilizada o Python de versão 3.10.4, conforme este script foi desenvolvido.

In [ ]:
# Importação das Bibliotecas
from IPython.display import clear_output
from datetime import datetime
import geopandas as gpd
import rasterio.mask
import numpy as np
import rasterio
import os
gpd.options.use_pygeos = True

## Declaração dos Paths

Aqui, fazemos uma simples declaração dos caminhos dos arquivos que serão utilizados para a inserção das features. Para fins de otimização, o dataframe dos eventos já será aberto nesta seção.

In [ ]:
# Path dos Eventos
fire_events_path = '/d01/scholles/Codigos/teste/samples.shp'
saving_path = '/d01/scholles/Codigos/teste/samples.shp'

# Paths de todos os Shapefiles a serem utilizados
indigenous_areas_path = '/d01/scholles/Codigos/maps/areas_features/Reservas_Indigenas/tis_poligonais_single_parts.shp'
deter_amz_alerts_path = '/d01/scholles/Codigos/maps/areas_features/deter-amz-public-2022dez06/deter_public.shp'
conservation_units_path = '/d01/scholles/Codigos/maps/areas_features/Reservas_Ambientais/UNC_21_BR_CEM_V2_single_parts.shp'
#prodes_yearly_deforestation_path = '/d01/scholles/Codigos/maps/areas_features/deforestation_since_2007/deforestation_since_2007.shp'
private_mv_terra_path = '/d01/scholles/Codigos/maps/areas_features/mv_terra_legal_amazon/privado/private_mv_terra_legal_amazon.shp'
unknown_mv_terra_path = '/d01/scholles/Codigos/maps/areas_features/mv_terra_legal_amazon/unknown/unknown_mv_terra_legal_amazon.shp'
public_mv_terra_path = '/d01/scholles/Codigos/maps/areas_features/mv_terra_legal_amazon/publico/public_mv_terra_legal_amazon.shp'
car_shape_path = '/d01/scholles/Codigos/maps/areas_features/tb_car_limites/legal_amazon/tb_car_limites_legal_amazon.shp'

# Paths de todos os TIFFs a serem utilizados
brasil_coverage_path = '/d01/scholles/Codigos/maps/areas_features/brazil_coverage_2019/brasil_coverage_2019.tif'
hansen_tree_cover = '/d01/scholles/Codigos/maps/areas_features/hansen_tree_cover_2000/merged/hansen_tree_cover2000.tif'
above_ground_tiff_path = '/d01/scholles/Codigos/maps/areas_features/biomass_map/above_ground_biomass.tif'
growing_stock_tiff_path = '/d01/scholles/Codigos/maps/areas_features/biomass_map/growing_stock_volume.tif'

# Leitura do SHP dos Eventos
fire_events_gdf = gpd.GeoDataFrame.from_file(fire_events_path, geometry='geometry')

## Funções entre Shapefiles

Nesta seção, definiremos as funções que serão utilizadas para gerar a interseção entre os Shapefiles dos eventos de fogo e diversas áreas de interesse.

### Função Geral de Overlay

A primeira função a ser definida é a de Overlay. Conforme mostra a imagem de exemplo abaixo, geramos um shapefile por meio ta intercessão dos eventos de fogo com outro shapefile de interesse. No momento, esse overlay é utilizado para gerar features referentes à:

<dd>• Incremento anual no desmatamento PRODES (2008-2022)</dd>
<dd>• Avisos na Amazônia Legal DETER (desde 2016)</dd>
<dd>• Áreas do Cadastro Ambiental Rural (CAR)</dd>
<dd>• Áreas Públicas, Privadas, Desconhecido</dd>
<dd>• Unidades de Conservação Ambiental</dd>
<dd>• Áreas Indígenas</dd>

Para utilizar tal função, basta o usuário fornecer o path de ambos os shapefiles (evento e área de interesse). Tal função foi implementada utilizando a função geopandas.overlay(). Ela funciona basicamente gerando sobreposições espaciais entre dois GeoDataFrames para extração das features, posteriormente. Abaixo temos um exemplo de overlay (cor marrom) entre polígonos dos eventos de fogo (cor verde) e de Terras Indígenas (cor vermelha).

![image.png](https://cdn.discordapp.com/attachments/747881244544598067/1051649698789859380/image.png)

In [ ]:
def overlay_two_shps(event_gdf, path_interest_shp):
    # Faz a leitura do shape que fará o overlay
    interest_gdf = gpd.GeoDataFrame.from_file(path_interest_shp, geometry='geometry')    
    
    # Configura o CRS do evento para fazer a interseção entre ambos os SHPs
    try:
        # Verifica se as camadas já estão em CRS 4326
        if not event_gdf.crs or event_gdf.crs.to_epsg() != 4326:
            event_gdf = event_gdf.to_crs(crs=4326)
        
        if not interest_gdf.crs or interest_gdf.crs.to_epsg() != 4326:
            interest_gdf = interest_gdf.to_crs(crs=4326)
        
        # Verifica se as geometrias são válidas antes de tentar a interseção
        if not event_gdf.is_valid.all():
            event_gdf = event_gdf[event_gdf.is_valid]
        
        if not interest_gdf.is_valid.all():
            interest_gdf = interest_gdf[interest_gdf.is_valid]
    
        # Realiza o overlay
        overlayed_shp = gpd.overlay(interest_gdf, event_gdf, how='intersection')
    
    except Exception as e:
        print(f"Erro ao realizar a interseção: {str(e)}")    

    # Converte as coordenadas para metros, para assim calcularmos a área dos polígonos
    aux_gdf = overlayed_shp.to_crs(crs=3857)
    inter_area = aux_gdf.area.to_list()
    inter_area = list(map(lambda x: x/1000000, inter_area))

    # Passa o valor da área em km2 para cada uma dos polígonos
    overlayed_shp['pol_area'] = inter_area

    # Deleta as variáveis que não possuem mais utilidade
    del interest_gdf, aux_gdf, inter_area

    return overlayed_shp

### Função extra para filtragem do overlay do DETER

Para os alertas de desmatamento do DETER, serão geradas 2 features que serão melhor detalhadas posteriormente, mas, em suma, algumas features deverão atender o quesito de que os alertas de desmatamento tenham acontecido em no máximo 400 dias antes do evento de fogo. Outra coisa que é importante de ser filtrada são os alertas que desmatamento que ocorreram depois do evento de fogo, que devem ser eliminados. Assim, a função abaixo faz justamente esta filtragem.

In [ ]:
def deter_filter(deter_overlay_gdf):
    
    # Cria uma variável auxiliar para manipular o GeoDataframe
    deter_overlay_gdf_aux = deter_overlay_gdf
    
    # Lista para receber a diferença de tempo no qual ocorreu entre o evento
    # de fogo e o alerta de desmatamento do DETER
    delta_time = []

    # Cria a lista que receberá os índices que deverão ser eliminados após
    # a aferição dos alertas que não se enquadram nos critérios
    indexes_out_400range = []
    indexes_alerts_after = []    
    
    # Transforma as colunas das datas dos alertas de desmatamento e
    # eventos de fogo em listas, para uma melhor otimização
    desm_dates = deter_overlay_gdf_aux['VIEW_DATE'].to_list()
    event_dates = deter_overlay_gdf_aux['dt_minima'].to_list()

    for i in range(len(event_dates)):
        # Pega a data do evento e subtrai pela data do alerta de desmatamento
        try:
            event_date = datetime.strptime(event_dates[i][:10], '%Y-%m-%d')            
        except:
            event_date = event_dates[i]
            
        desm_date = datetime.strptime(desm_dates[i], '%Y-%m-%d')    
        delta = event_date - desm_date
        delta = delta.days
        delta_time.append(delta)

        # Para casos em que o alerta de desmtamento aconteceu depois do
        # evento de fogo, capturamos seus índices para depois os deletarmos
        if 0 > delta:
            indexes_alerts_after.append(i)

        # Para casos em que o alerta de desmtamento aconteceu depois do
        # evento de fogo ou antes de 400 dias, capturamos seus índices
        # para depois os deletarmos
        if 0 > delta or delta > 400:
            indexes_out_400range.append(i)

    # Aplica ambos os filtros, de acordo com as necessidades especificadas
    deter_overlay_gdf_aux['delta_date'] = delta_time
    deter_simple_filtered_gdf = deter_overlay_gdf_aux.drop(indexes_alerts_after)
    deter_filtered_400 = deter_overlay_gdf_aux.drop(indexes_out_400range)

    # Reseta o índice dos GeoDataframes
    deter_simple_filtered_gdf = deter_simple_filtered_gdf.reset_index()
    deter_filtered_400 = deter_filtered_400.reset_index()

    # Deleta as variáveis que não possuem mais utilidade
    del deter_overlay_gdf_aux, indexes_out_400range, indexes_alerts_after, delta_time

    return deter_simple_filtered_gdf, deter_filtered_400

### Geração dos Overlays

Aqui, basicamente criamos os GeoDataframes para extrair as features desejadas para a subseção que virá mais adiante.

In [ ]:
# Geração dos overlays de interesse
print('Gerando o overlay de Unidades de Conservação Ambiental...')
conservation_units_overlay = overlay_two_shps(fire_events_gdf, conservation_units_path)

print('Gerando o overlay de Áreas Indígenas...')
indigenous_areas_path_overlay = overlay_two_shps(fire_events_gdf, indigenous_areas_path)

#print('Gerando o overlay do PRODES...')
#prodes_yearly_deforestation_overlay = overlay_two_shps(fire_events_gdf, prodes_yearly_deforestation_path)

print('Gerando o overlay do DETER...')
deter_amz_alerts_overlay = overlay_two_shps(fire_events_gdf, deter_amz_alerts_path)

print('Gerando o overlay do CAR...')
CAR_overlay = overlay_two_shps(fire_events_gdf, car_shape_path)

print('Gerando o overlay de terras públicas, privadas e desconhecidas...')
private_mv_terra_overlay = overlay_two_shps(fire_events_gdf, private_mv_terra_path)
unknown_mv_terra_overlay = overlay_two_shps(fire_events_gdf, unknown_mv_terra_path)
public_mv_terra_overlay = overlay_two_shps(fire_events_gdf, public_mv_terra_path)

#Filtragem dos dados do DETER
print('Filtrando o overlay do DETER...')
deter_amz_alerts_overlay, deter_amz_alerts_overlay_only_400 = deter_filter(deter_amz_alerts_overlay)

### Extração das features dos SHPs

Nesta subseção, faremos a extração das features dos overlays gerados. Algumas funções abaixo serão válidas para múltiplos dos overlays, enquanto outras serão específicas para certas features. Assim, aqui citaremos as features que serão obtidas para cada uma dos shapefiles utilizados.

1. Incremento anual no desmatamento PRODES (2007/2022):

<dd>• Área (km²) no qual possui histórico de desmatamento dentro da área do evento/polígono (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

2. Avisos na Amazônia Legal DETER (desde 2016)

<dd>• Quantidade de dias que houve um alerta de desmatamento na área do evento/polígono (int)</dd>
<dd>• Área (km²) dos alertas de desmatamento naquele evento/polígono, nos últimos 400 dias (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

3. Unidades de Conservação Ambiental

<dd>• Área (km²) no qual o evento/polígono está inserido dentro de uma Unidade de Conservação Ambiental (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

4. Áreas Indígenas

<dd>• Área (km²) no qual o evento/polígono está inserido dentro de uma Área Indígena (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

5. Áreas Públicas, Privadas, Desconhecido

<dd>• Área (km²) no qual o evento/polígono está inserido dentro de Áreas Públicas, Privadas e Desconhecidas (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

6. Áreas do Cadastro Ambiental Rural (CAR)

<dd>• Área (km²) no qual o evento/polígono está inserido dentro de Áreas do Cadastro Ambiental Rural (float)</dd>
<dd>• Percentual que ocupa a área acima, dentro do evento/polígono (float)</dd>

Assim, abaixo temos nossa primeira função, referente ao cálculo da área no qual tal feature representa dentro daquele evento/polígono. Em seguida, temos uma outra função que funciona de forma similar ao cálculo da área, porém é exclusiva para calcular o tempo em dias que houve um alerta de desmatamento dentro do overlay do DETER.

In [ ]:
def calculate_area_from_overlay(event_gdf, overlay_gdf, area_name):
    # Nome da área de cada polígono formado na operação de overlay 
    polygon_area = 'pol_area'

    new_areas = [0] * len(event_gdf)

    # Forma de busca e cruzamento de dados é feita pelo ID do evento
    intersections_ids = overlay_gdf['id_evento'].tolist()
    area_inside = overlay_gdf[polygon_area].tolist()

    for i in range(len(intersections_ids)):
        
        # Pega todo os polígonos formados no overlay e os soma, baseado no ID dos eventos
        index = event_gdf.loc[(event_gdf['id_evento'] == int(intersections_ids[i]))].index[0]
        total_Area_In = sum(overlay_gdf.loc[(overlay_gdf['id_evento'] == int(intersections_ids[i]))][polygon_area].to_list())
        new_areas[index] = total_Area_In

    event_gdf[area_name] =  new_areas

    del intersections_ids, area_inside, new_areas

    return event_gdf

def calculate_delta_time_from_deter(event_gdf, overlay_gdf):
    
    # Forma de busca e cruzamento de dados é feita pelo ID do evento e geração da lista para saber o dia mais
    # recente que houve desmatamento na área
    min_delta_time = [9999] * len(event_gdf)
    intersections_ids = overlay_gdf['id_evento'].tolist()

    for i in range(len(intersections_ids)):
        
        #Observa, a partir do ID, qual foi a data mais recente que houve um alerta de desmatamento
        index = event_gdf.loc[(event_gdf['id_evento'] == int(intersections_ids[i]))].index[0]
        most_recent_date = min(overlay_gdf.loc[(overlay_gdf['id_evento'] == int(intersections_ids[i]))]['delta_date'].to_list())        
        min_delta_time[index] = most_recent_date

    event_gdf['deter_days'] =  min_delta_time

    # Deleta as variáveis que não possuem mais utilidade
    del min_delta_time, intersections_ids

    return event_gdf

Em seguida, utilizamos a seguinte função para calcular o percentual que ocupam as features de área anteriores, dentro do evento/polígono.

In [ ]:
def percentage_area_in(event_gdf, area_name, percentage_area_name):

    total_area = event_gdf['area_km2'].tolist()
    target = event_gdf[area_name].tolist()

    # Divide os termos de X por Y (através do seu respectivo index)
    percentage_array = [x / y for x, y in zip(target, total_area)]  
    
    # Tudo que for maior que 1, ele arredonda pra 1, já que os cálculo da área
    # das features não é 100% preciso
    percentage_array = [1 if i > 1 else i for i in percentage_array] 

    event_gdf[percentage_area_name] = percentage_array

    # Deleta as variáveis que não possuem mais utilidade
    del percentage_array, total_area, target

    return event_gdf

### Geração das Features

Aqui, basicamente chamamos as funções que geraram as features mencionadas anteriormente.

In [ ]:
# Geração das features de Unidades de Conservação Ambiental
print('Gerando as features de Unidades de Conservação Ambiental')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, conservation_units_overlay, 'areakm2_UC')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'areakm2_UC', 'perc_in_UC')

# Geração das features de Áreas Indígenas
print('Gerando as features de Áreas Indígenas')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, indigenous_areas_path_overlay, 'areakm2_IN')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'areakm2_IN', 'perc_in_IN')

# Geração das features do PRODES
print('Gerando as features do PRODES')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, prodes_yearly_deforestation_overlay, 'area_desm')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'area_desm', 'perc_desm')

print('Gerando as features de Terras Privadas')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, private_mv_terra_overlay, 'a_terr_pri')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'a_terr_pri', 'p_terr_pri')

print('Gerando as features de Terras Publicas')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, public_mv_terra_overlay, 'a_terr_pub')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'a_terr_pub', 'p_terr_pub')

print('Gerando as features de Terras Desconhecidas')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, unknown_mv_terra_overlay, 'a_terr_unk')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'a_terr_unk', 'p_terr_unk')

print('Gerando as features do CAR...')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, CAR_overlay, 'area_CAR')
fire_events_gdf = percentage_area_in(fire_events_gdf, 'area_CAR', 'per_CAR')

# Geração das features do DETER
print('Gerando as features do DETER')
fire_events_gdf = calculate_area_from_overlay(fire_events_gdf, deter_amz_alerts_overlay_only_400, 'deter_area')
fire_events_gdf = calculate_delta_time_from_deter(fire_events_gdf, deter_amz_alerts_overlay)
fire_events_gdf = percentage_area_in(fire_events_gdf, 'deter_area', 'deter_perc')

print('Finalizada a inserção das features baseadas em Shapefiles')

## Funções paras os GeoTIFFs

Nesta seção, definiremos as funções que serão utilizadas para gerar a interseção entre os Shapefiles dos eventos de fogo e os GeoTIFFs e geração de novas features. Assim, nesta seção, trabalharemos com features referentes à:

<dd>• Cobertura de Árvores</dd>
<dd>• Uso do Solo</dd>
<dd>• Biomassa</dd>

### Função para Uso do Solo

Nesta subseção, temos uma função que insere informações de área e percentual que cada tipo de solo, mapeado pelo MapBiomas, cobrem cada evento de fogo/polígono. Vale lembrar que nem todos os tipos de solo presentes neste mapeamento estão presentes na Amazônia Legal.

![alternatvie text](https://cdn.discordapp.com/attachments/747881244544598067/1051637628010766426/image.png)

A legenda para cada tipo de solo é:

<dd>• Tipo 0:  "Não Observado"</dd>
<dd>• Tipo 1:  "Floresta"</dd>
<dd>• Tipo 2:  "Floresta Natural"</dd>
<dd>• Tipo 3:  "Formação Florestal"</dd>
<dd>• Tipo 4:  "Formação da Savana"</dd>
<dd>• Tipo 5:  "Magrove"</dd>
<dd>• Tipo 6:  "Áreas Naturales Inundables - Leñosas (Bosque Inundable)"</dd>
<dd>• Tipo 9:  "Plantação Florestal"</dd>
<dd>• Tipo 10: "Formação Natural Não Florestal"</dd>
<dd>• Tipo 11: "Terra molhada"</dd>
<dd>• Tipo 12: "Grassland (Pastizal Formación Herbácea)"</dd>
<dd>• Tipo 13: "Outras Formações Naturais Não Florestais"</dd>
<dd>• Tipo 14: "Agricultura"</dd>
<dd>• Tipo 15: "Pastagens"</dd>
<dd>• Tipo 18: "Agricultura"</dd>
<dd>• Tipo 19: "Culturas temporárias (Herbaceas - Agricultura)"</dd>
<dd>• Tipo 20: "Cana-de-açúcar"</dd>
<dd>• Tipo 21: "Mosaico de Agricultura e Pastagem"</dd>
<dd>• Tipo 22: "Área não vegetada"</dd>
<dd>• Tipo 23: "Praia e Duna"</dd>
<dd>• Tipo 24: "Infra-estruturas urbanas"</dd>
<dd>• Tipo 25: "Outra Área Não Vegetada"</dd>
<dd>• Tipo 26: "Água"</dd>
<dd>• Tipo 27: "Não Observado"</dd>
<dd>• Tipo 29: "Outcrop rochoso"</dd>
<dd>• Tipo 30: "Mineração"</dd>
<dd>• Tipo 31: "Aquacultura"</dd>
<dd>• Tipo 32: "Sal plano"</dd>
<dd>• Tipo 33: "Rio Lago e Oceano"</dd>
<dd>• Tipo 35: "Palma de óleo"</dd>
<dd>• Tipo 36: "Culturas Perenes"</dd>
<dd>• Tipo 37: "Corpo de Água Artificial"</dd>
<dd>• Tipo 38: "Reservatórios de água"</dd>
<dd>• Tipo 39: "Feijão de Soja"</dd>
<dd>• Tipo 40: "Arroz"</dd>
<dd>• Tipo 41: "Mosaico de colheitas"</dd>
<dd>• Tipo 42: "Pastizal abierto"</dd>
<dd>• Tipo 43: "Pastizal cerrado"</dd>
<dd>• Tipo 44: "Pastizal disperso"</dd>
<dd>• Tipo 45: "Leñosas dispersas"</dd>
<dd>• Tipo 46: "Café"</dd>
<dd>• Tipo 47: "Citrus"</dd>
<dd>• Tipo 48: "Outras Culturas Perenes"</dd>
<dd>• Tipo 49: "Vegetação de banco de areia arborizado"</dd>
<dd>• Tipo 50: "Vegetação Herbácea de banco de areia"</dd>
<dd>• Tipo 57: "Cultivo Simples"</dd>
<dd>• Tipo 58: "Cultivo Múltiplo"</dd>
<dd>• Tipo 62: "Algodão"</dd>

In [ ]:
def coverage_features(event_gdf, tiff_path):
    # Criação de um array que abrigará todos os arrays referentes ao uso de solo e seu percentual
    area_root_array = [] 
    perc_root_array = []

    # Declaração dos 63 arrays que receberão append
    for i in range(63):
        area_root_array.append([]) 
        perc_root_array.append([])

    # Como desconsideramos os pixels de valor 0, já os setamos todos como 0
    area_root_array[0] = [0] * len(event_gdf)
    perc_root_array[0] = [0] * len(event_gdf)

    # Abre o raster de entrada e recorta o raster de saída
    with rasterio.open(tiff_path) as src:
        for index in range(len(event_gdf)):
            # Índice do polígono que está sendo analisado
            polygon = event_gdf.loc[index]  

            out_image, out_transform = rasterio.mask.mask(src, [polygon["geometry"]], crop=True)
            out_meta = src.meta
            out_meta.update({
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,}
            )

            for j in range(63):
                if j != 0:
                    contagem_pixels = (out_image == j).sum()
                    area_pixels = (contagem_pixels * 900)/1000000                    
                    area_root_array[j].append(round(area_pixels,5))
                    perc_root_array[j].append(round((area_pixels/polygon["area_km2"]),4))

            if index % 25000 == 0:
                print('Porcentagem da interseção (Uso do Solo):', (round(((index/len(event_gdf))*100),2)), '%.')
                clear_output()
                print('Porcentagem da interseção (Uso do Solo):', (round(((index/len(event_gdf))*100),2)), '%.')

    # Passagem das features extraídas para o GeoDataFrame
    for i in range(63):
        area_column_name = 'a_cover_' + str(i)
        perc_column_name = 'p_cover_' + str(i)
        event_gdf[area_column_name] = area_root_array[i]
        event_gdf[perc_column_name] = perc_root_array[i]
    
    clear_output()

    return event_gdf

### Função para média dos valores de um TIFF dentro de um polígono

De forma muito similar à anterior, temos uma função que insere informações do percentual médio dos valores do TIFF dentro da área delimitada pelo polígono (i.e.: média da cobertura de árvores na área do polígono/evento de fogo)

In [ ]:
def mean_overlay_tiff_shp(event_gdf, tiff_path, feature_name):

    feature_array = []   

    # Abre o raster de entrada e recorta o raster de saída
    with rasterio.open(tiff_path) as src:
        for index in range(len(event_gdf)):
            polygon = event_gdf.loc[index]  #Índice do polígono que está sendo analisado

            out_image, out_transform = rasterio.mask.mask(src, [polygon["geometry"]], crop=True, nodata= -1)
            out_meta = src.meta
            out_meta.update({
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,}
            )

            # Transforma a matriz da imagem num array de uma dimensão
            pixels_values = out_image.ravel()        
            
            # Pra casos em que o polígono está no rio ou mar, ou seja, 0
            if len(np.unique(pixels_values)) == 1 and np.unique(pixels_values)[0] == -1:       
                feature_mean = 0
                feature_array.append(feature_mean)
            else:                
                pixels_values = pixels_values[pixels_values != -1] # Desconsidera pixels fora da borda do polígono
                feature_mean = pixels_values.mean()        
                feature_array.append(feature_mean)
            
            if index % 25000 == 0:
                print('Porcentagem da interseção (Cobertura de Árvores):', (round(((index/len(event_gdf))*100),2)), '%.')
                clear_output()
                print('Porcentagem da interseção (Cobertura de Árvores):', (round(((index/len(event_gdf))*100),2)), '%.')

    event_gdf[feature_name] = feature_array

    return event_gdf

### Execução das funções

Por fim, executamos as funções para a geração das features provindas dos TIFFs.

In [ ]:
# Executa ambas as funções
fire_events_gdf = mean_overlay_tiff_shp(fire_events_gdf, growing_stock_tiff_path, 'biomass_GS')
fire_events_gdf = mean_overlay_tiff_shp(fire_events_gdf, above_ground_tiff_path, 'biomass_AG')
fire_events_gdf = mean_overlay_tiff_shp(fire_events_gdf, hansen_tree_cover, 'tree_cover')
fire_events_gdf = coverage_features(fire_events_gdf, brasil_coverage_path)

print('Finalizadas as funções para os GeoTIFFs')

## Funções Extras

### Função de filtragem de colunas inutilizadas

Quando geramos os dados coletados acima, algumas colunas ficam vazias ou possuem apenas um valor na coluna, o que as torna obsoletas na hora do treinamento e/ou predição. Para isso, então, podemos utilizar a seguinte função, com finalidade de limpar tais colunas:

In [ ]:
def filter_unused_columns(event_gdf):

    # Todas as colunas de atributos shapefile
    columns = event_gdf.columns.to_list()
    unused_columns = []

    for i in columns:
        # Não faz a checagem na coluna de geometria
        if i != 'geometry':
            # Salva todos os valores únicos da coluna
            column = np.array(event_gdf[i].to_list())
            unique_numbers = np.unique(column)

            # Caso a coluna possua apenas um valor único, salvamos seu nome para
            # removermos posteriormente
            if len(unique_numbers) == 1:
                unused_columns.append(i)

    # Remove as colunas que foram consideradas obsoletas
    for i in unused_columns:
        columns.remove(i)

    # Mantém apenas as colunas que possuem diversos valores
    event_gdf = event_gdf[columns]

    return event_gdf

## Salvando o GeoDataframe final

Por fim, nesta seção, rodamos os últimos scripts de filtragem do dataset e com isso salvamos o arquivo, a partir de um path definindo onde será salvo o shapefile com todas as features acrescentadas.

In [ ]:
# Executa a filtragem das colunas que não são úteis
fire_events_gdf = filter_unused_columns(fire_events_gdf)

# Salva o shape com todas as features
fire_events_gdf.to_file(saving_path)